In [13]:
import time
import random
import collections
import numpy as np
from causality_functions import jitter, dependent, simulations_data
from causality_functions import compute_regression, compute_correlation
from causality_functions import add_confidence_stats, confidence_graphs
from causality_functions import stats_graphs
from causality_functions import check_generation

import altair as alt
import pandas as pd
import plotly.express as px
alt.data_transformers.disable_max_rows()

samplesize = 500
Am, Bm, Cm, = 40, 10, 30
Xm, Ym = 1, 1
Astd,Bstd,Cstd = 5, 13, 12
Xstd,Ystd = 1, 1

m1, k1 = 1, 0
m2, k2 = 1.5, 0
m3, k3 = 0.15, 0 # this is feedback term
m4, k4 = 0.75, 0.0

dg = 0.1
d1 = 1
d2 = 1
d3 = 1
d4 = 1

errorA = 2
errorB = 4
errorC = 3.5
errorX = 0.75
errorY = 0.6

mrange1 = 0,5 #slope ranges
mrange2 = 0,5
mrange3 = 0,5
mrange4 = 0,5

erangeA = 0,1 #error range
erangeB = 0.2,10 #error range
erangeC = 0.2,10 #error range
erangeX = 0.2,5 #error range
erangeY = 0.2,5 #error range


def linear_positive_C_to_B(n=100):

    def compute_B(history, generation):
        if check_generation(generation, d1):
             A, _, _ = history[1 - d1]   
        else:
             A, _, _ = history[-1]
        if check_generation(generation, d3):
            _, _, C = history[1 - d3]
        else:
            _, _, C = history[-1]
        _, B, _ = history[-1]
        return jitter(B + A*m1 + C*m3 - B*m2 -dg*B + k1 ,  errorB)
        

    def compute_C(history, generation):
        if check_generation(generation, d2):
            _, B, _ = history[1 - d2]
        else:
            _, B, _ = history[-1]
        _, _, C = history[-1]
        return jitter(C + B*m2 - dg*C + k2, errorC)

    def next_generation(history, generation):
        A, _, _ = history[-1]
        A_ = jitter(A, errorA) # previous A + error

        # compute B
        B_ = compute_B(history, generation)
        # compute C
        C_ = compute_C(history, generation)
        
        return A_, B_, C_

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    history = collections.deque(maxlen=max([d1, d2, d3]) + 1)
    history.append((A, 0, 0))
    for i in range(n + max([d1, d2, d3])):
        history.append(next_generation(history, i))

    return np.array(history[-1])


def linear_positive_A_to_C(n=100):    

    
    def compute_A(history, generation):
        A, _, _ = history[-1]
        return jitter(A, errorA)
        
    def compute_B(history, generation):
        if check_generation(generation, d1):
            A, _, _ = history[1 - d1]
        else:
            A, _, _ = history[-1]
            
        return dependent(A, m1, k1 ,  errorB)
        

    def compute_C(history, generation):
        if check_generation(generation, d1):
            A, _, _ = history[1 - d1]
        else:
            A, _, _ = history[-1]
        
        if check_generation(generation, d2):
            _, B, _ = history[1 - d2]
        else:
            _, B, _ = history[-1]
            
        _, _, C = history[-1]
        return jitter(A*m3+ B*m2+ -dg*C + k2, errorC)
        
            
    
    def next_generation(history, generation):
        A_ = compute_A(history, generation)
        B_ = compute_B(history, generation)
        C_ = compute_C(history, generation)
        return A_, B_, C_

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    history = collections.deque(maxlen=max([d1, d2, d3]) + 1)
    history.append((A, 0, 0))
    for i in range(n + max([d1, d2, d3])):
        history.append(next_generation(history, i))

    return np.array(history[-1])

def update_slopes_errors():
    def select(lower, upper):
        return lower + random.random()*(upper-lower)
    
    global m1, m2, m3, m4
    global errorA, errorB, errorC, errorX, errorY

    m1 = select(*mrange1)
    m2 = select(*mrange2)
    m3 = select(*mrange3)
    m4 = select(*mrange4)
    
    errorA = select(*erangeA)
    errorB = select(*erangeB)
    errorC = select(*erangeC)
    errorX = select(*erangeX)
    errorY = select(*erangeY)
    
def overall_simulation(n=100):
    stats = []
    data = []
    for i in range(n):
        update_slopes_errors()
        ABC = simulations_data(linear_positive_C_to_B, samplesize) # you can change pathway here
        r = compute_regression(ABC)
        r.update(compute_correlation(ABC))
        stats.append(r)
        data.append(ABC)
    return pd.DataFrame(stats), np.array(data)

d, ABC_all = overall_simulation()

In [14]:
add_confidence_stats(d, ABC_all)
confidence_graphs(d)

alt.VConcatChart(...)

In [15]:
stats_graphs(d)

alt.VConcatChart(...)

In [16]:
A, B, C = random.choice(ABC_all).transpose()
AB = alt.Chart(pd.DataFrame({"A":A, "B":B})).mark_circle().encode(
    x="A",
    y="B")
BC= alt.Chart(pd.DataFrame({"B":B, "C":C})).mark_circle().encode(
    x="B",
    y="C")
AC = alt.Chart(pd.DataFrame({"A":A, "C":C})).mark_circle().encode(
    x="A",
    y="C")

alt.vconcat(AB, BC, AC)

alt.VConcatChart(...)

In [12]:
d

,kAB,kBC,kAC,mAB,mBC,mAC,r_sqrAB,r_sqrBC,r_sqrAC,r_E,r_E_BA_C,n,rAB,rBC,rAC,rAB2*rBC2-rAC2,r_E_BA_C2-rBC2,mAB*mBC-mAC,confidence_rAC,confidence_residual_corr,confidence_corrected_bc_corr,confidence_slope_AC
0,-0.273865,26.030552,2.336562,0.208463,0.168228,0.623959,0.054262,0.005995,0.102981,-0.072765,0.016360,500,0.232942,0.077429,0.320907,-0.102656,-0.005728,-0.588890,less,within,within,within
1,0.966195,35.266628,11.624089,2.401749,3.636323,9.410034,0.912727,0.864007,0.915504,-0.603222,0.040713,500,0.955368,0.929520,0.956820,-0.126901,-0.862349,-0.676497,less,more,less,within
2,1.406286,123.746075,1.870615,0.392518,1.989325,3.928367,0.194632,0.180798,0.890634,-0.410696,-0.093678,500,0.441171,0.425203,0.943734,-0.855445,-0.172022,-3.147521,less,more,less,within
3,2.603431,24.627197,8.228026,4.564820,2.393252,11.486206,0.946271,0.906055,0.947762,-0.651545,0.033638,500,0.972765,0.951869,0.973531,-0.090388,-0.904923,-0.561443,less,more,less,within
4,1.904371,70.105346,29.024915,4.149184,3.559431,15.959406,0.912680,0.853107,0.909220,-0.600085,0.009347,500,0.955343,0.923638,0.953530,-0.130606,-0.853019,-1.190673,less,more,less,within
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.929177,46.614014,-1.677342,1.117066,4.195302,6.109358,0.773074,0.697932,0.916941,-0.748988,-0.061377,500,0.879246,0.835423,0.957571,-0.377388,-0.694165,-1.422930,less,more,less,within
96,-0.116855,15.127238,1.320118,0.450013,8.504560,4.143243,0.910839,0.920818,0.982975,-0.859108,-0.029865,500,0.954379,0.959593,0.991451,-0.144258,-0.919926,-0.316080,less,more,less,within
97,-1.864198,2.635662,1.191303,4.990952,0.316912,1.603227,0.987861,0.477719,0.484859,-0.116755,-0.050258,500,0.993912,0.691173,0.696318,-0.012939,-0.475194,-0.021536,within,more,less,within
98,0.951117,83.086711,-12.079918,1.581858,1.001408,3.975515,0.390294,0.202571,0.497966,-0.380324,-0.042504,500,0.624735,0.450079,0.705667,-0.418904,-0.200765,-2.391430,less,more,less,within
